In [1]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

2023-06-17 13:25:21.898694: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-17 13:25:22.270323: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.


KeyboardInterrupt: 

# Specify each path

In [ ]:
dataset = 'model/keypoint_classifier/keypoint.csv'
model_save_path = 'model/keypoint_classifier/keypoint_classifier.hdf5'
tflite_save_path = 'model/keypoint_classifier/keypoint_classifier.tflite'

# Set number of classes

In [ ]:
NUM_CLASSES = 10

# Dataset reading

In [ ]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))

In [ ]:
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)

# Model building

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(80, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [ ]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 42)                0         
                                                                 
 dense (Dense)               (None, 80)                3440      
                                                                 
 dropout_1 (Dropout)         (None, 80)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                810       
                                                                 
 dense_2 (Dense)             (None, 10)                110       
                                                                 
Total params: 4,360
Trainable params: 4,360
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [ ]:
# Model compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Model training

In [ ]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
46/52 [=========================>....] - ETA: 0s - loss: 2.1807 - accuracy: 0.2193 
Epoch 1: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
52/52 [==============================] - 1s 5ms/step - loss: 2.1557 - accuracy: 0.2312 - val_loss: 1.9331 - val_accuracy: 0.3469
Epoch 2/1000
46/52 [=========================>....] - ETA: 0s - loss: 1.8393 - accuracy: 0.3405
Epoch 2: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
52/52 [==============================] - 0s 2ms/step - loss: 1.8305 - accuracy: 0.3405 - val_loss: 1.6749 - val_accuracy: 0.3528
Epoch 3/1000
45/52 [========================>.....] - ETA: 0s - loss: 1.6295 - accuracy: 0.3903
Epoch 3: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
52/52 [==============================] - 0s 2ms/step - loss: 1.6203 - accuracy: 0.3942 - val_loss: 1.4501 - val_accuracy: 0.5317
Epoch 4/1000
33/52 [==================>...........] - ETA: 0s - loss: 1.4804 - accuracy: 0.4536


In [ ]:
# Model evaluation
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

18/18 [==============================] - 0s 916us/step - loss: 0.2402 - accuracy: 0.9530


In [ ]:
# Loading the saved model
model = tf.keras.models.load_model(model_save_path)

In [ ]:
# Inference test
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

1/1 [==============================] - 0s 57ms/step
[8.8255582e-03 9.6602784e-04 1.2036941e-02 1.1883562e-10 2.7194763e-12
 6.9666987e-07 2.7538767e-02 7.7406412e-01 1.7656749e-01 4.6999349e-07]
7


# Saving Model

In [ ]:
# Save as a model dedicated to inference
model.save(model_save_path, include_optimizer=False)

In [ ]:
# Transform model (quantization)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

2023-06-12 08:28:30.630710: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_1' with dtype float and shape [?,42]
	 [[{{node input_1}}]]
2023-06-12 08:28:30.682699: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,42]
	 [[{{node inputs}}]]
2023-06-12 08:28:30.694221: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_1' with dtype float and shape [?,42]
	 [[{{node input_1}}]]
2023-06-12 08:2

INFO:tensorflow:Assets written to: /tmp/tmplkg0v6pn/assets


INFO:tensorflow:Assets written to: /tmp/tmplkg0v6pn/assets
2023-06-12 08:28:31.630351: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-06-12 08:28:31.630408: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-06-12 08:28:31.631300: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmplkg0v6pn
2023-06-12 08:28:31.632377: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-06-12 08:28:31.632393: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmplkg0v6pn
2023-06-12 08:28:31.635837: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
2023-06-12 08:28:31.636773: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-06-12 08:28:31.672776: I tensorflow/cc/saved_model/loader.cc:215] Running initializatio

9552

# Inference test

In [ ]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [ ]:
# Get I / O tensor
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

In [ ]:
%%time
# Inference implementation
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

CPU times: user 1.25 ms, sys: 92 µs, total: 1.34 ms
Wall time: 5.79 ms


In [ ]:
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))

[7.9772957e-03 8.1127422e-04 1.1780136e-02 1.1979551e-10 2.6501431e-12
 6.1536321e-07 2.7638040e-02 7.8212702e-01 1.6966517e-01 3.9018295e-07]
7
